### Renaming columns

In [61]:
import matplotlib.pyplot as plt
# opencv
import cv2
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten, GlobalAveragePooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, VGG16, EfficientNetB0


In [11]:


# Define paths
train = r"D:\fruits\fruits-360_original-size\fruits-360-original-size\train"
val = r"D:\fruits\fruits-360_original-size\fruits-360-original-size\val"
test = r"D:\fruits\fruits-360_original-size\fruits-360-original-size\test"

# Get class names
train_classes = set(os.listdir(train))
val_classes = set(os.listdir(val))
test_classes = set(os.listdir(test))

# Find common and mismatched class names
common_classes = train_classes & val_classes & test_classes
train_only = train_classes - common_classes
val_only = val_classes - common_classes
test_only = test_classes - common_classes

# Print summary
print(f" Common classes in all sets: {len(common_classes)}\n")
print(f" Classes only in train: {train_only}")
print(f" Classes only in val: {val_only}")
print(f" Classes only in test: {test_only}")






 Common classes in all sets: 15

 Classes only in train: set()
 Classes only in val: set()
 Classes only in test: set()


In [26]:
import os
import re

folder = r"D:\fruits\fruits-360_original-size\fruits-360-original-size\train"

for class_folder in os.listdir(folder):
    old_path = os.path.join(folder, class_folder)
    
    # Remove trailing underscores and numbers (e.g., 'carrot_1' → 'carrot')
    new_name = re.sub(r'[_\s]?\d+$', '', class_folder.lower())
    new_path = os.path.join(folder, new_name)

    if old_path == new_path:
        continue  # Already correct

    if not os.path.exists(new_path):
        os.rename(old_path, new_path)
        print(f"Renamed: {class_folder} → {new_name}")
    else:
        print(f"⚠️ Skipped (already exists): {new_name}")


Renamed: Apple 5 → apple
Renamed: Avocado Black 1 → avocado black
Renamed: Beans 1 → beans
Renamed: Blackberrie 1 → blackberrie
Renamed: Cabbage red 1 → cabbage red
Renamed: Cactus fruit green 1 → cactus fruit green
Renamed: Caju seed 1 → caju seed
Renamed: carrot_1 → carrot
Renamed: Cherimoya 1 → cherimoya
Renamed: Cherry 3 → cherry
Renamed: Cucumber 4 → cucumber
Renamed: eggplant_long_1 → eggplant_long
Renamed: Gooseberry 1 → gooseberry
Renamed: Nut 1 → nut
Renamed: pear_1 → pear


In [31]:

dataset_dir = r'D:\fruits\fruits-360_original-size\fruits-360-original-size\train'  # change to val or test if needed

total_images = 0

# Loop through each class folder
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        image_count = len([
            img for img in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, img))
        ])
        print(f"{class_name}: {image_count} images")
        total_images += image_count

print(f"\nTotal images in '{dataset_dir}': {total_images}")

apple: 294 images
avocado black: 468 images
beans: 155 images
blackberrie: 300 images
cabbage red: 99 images
cactus fruit green: 465 images
caju seed: 151 images
carrot: 101 images
cherimoya: 648 images
cherry: 465 images
cucumber: 152 images
eggplant_long: 160 images
gooseberry: 310 images
nut: 161 images
pear: 326 images

Total images in 'D:\fruits\fruits-360_original-size\fruits-360-original-size\train': 4255


In [33]:

dataset_dir = r'D:\fruits\fruits-360_original-size\fruits-360-original-size\val'  # change to val or test if needed

total_images = 0

# Loop through each class folder
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        image_count = len([
            img for img in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, img))
        ])
        print(f"{class_name}: {image_count} images")
        total_images += image_count

print(f"\nTotal images in '{dataset_dir}': {total_images}")

apple: 146 images
avocado black: 234 images
beans: 78 images
blackberrie: 150 images
cabbage red: 50 images
cactus fruit green: 234 images
caju seed: 75 images
carrot: 50 images
cherimoya: 324 images
cherry: 234 images
cucumber: 76 images
eggplant_long: 80 images
gooseberry: 156 images
nut: 80 images
pear: 162 images

Total images in 'D:\fruits\fruits-360_original-size\fruits-360-original-size\val': 2129


In [32]:

dataset_dir = r'D:\fruits\fruits-360_original-size\fruits-360-original-size\test'  # change to val or test if needed

total_images = 0

# Loop through each class folder
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        image_count = len([
            img for img in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, img))
        ])
        print(f"{class_name}: {image_count} images")
        total_images += image_count

print(f"\nTotal images in '{dataset_dir}': {total_images}")

apple: 146 images
avocado: 231 images
beans: 77 images
blackberrie: 150 images
cabbage: 49 images
cactus: 231 images
caju: 75 images
carrot: 50 images
cherimoya: 320 images
cherry: 231 images
cucumber: 76 images
eggplant_long: 80 images
gooseberry: 154 images
nut: 80 images
pear: 162 images

Total images in 'D:\fruits\fruits-360_original-size\fruits-360-original-size\test': 2112


In [45]:
train_image_gen= ImageDataGenerator(
                              # normalize
                              rescale=1/255, 
                              rotation_range=30, 
                              width_shift_range=0.1, 
                              shear_range=0.9, 
                            zoom_range=0.2, 
                              horizontal_flip=True, 
                              height_shift_range=0.1)

In [46]:

train_img = train_image_gen.flow_from_directory(train,
                                              target_size = (224,224)
                                              , batch_size=16,
                                              class_mode = 'sparse')

Found 4255 images belonging to 15 classes.


In [57]:
val_image_gen = ImageDataGenerator(rescale=1/255)

In [58]:
val_image = val_image_gen.flow_from_directory(val,
                                              target_size = (224,224)
                                              , batch_size=16,
                                              class_mode = 'sparse')

Found 2129 images belonging to 15 classes.


In [49]:
test_image_gen = ImageDataGenerator(rescale=1./255)

test_img = test_image_gen.flow_from_directory(
    test,
        target_size=(224, 224),
        batch_size=16,
        class_mode='sparse',
        shuffle=False  # Important for consistent evaluation, especially with confusion matrix etc.
    )


Found 2112 images belonging to 15 classes.


In [53]:
img_size = (224, 224)
batch_size = 32
num_classes = len(os.listdir(train))
num_classes

15

In [50]:
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


Training while freezing base layers, no early stopping and few epochs

In [60]:
base = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
resnet_model = build_model(base)

for layer in base.layers:
    layer.trainable = False

resnet_model.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_resnet = resnet_model.fit(train_img, validation_data=val_image, epochs=2)


Epoch 1/2
266/266 ━━━━━━━━━━━━━━━━━━━━ 396s 1s/step - accuracy: 0.1049 - loss: 2.9179 - val_accuracy: 0.1522 - val_loss: 2.4729
Epoch 2/2
266/266 ━━━━━━━━━━━━━━━━━━━━ 360s 1s/step - accuracy: 0.1491 - loss: 2.6059 - val_accuracy: 0.2184 - val_loss: 2.4278


Unfreezing base layers, early stopping and more epochs

In [ ]:
# Freeze all base layers first

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, verbose=1)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

resnet_model_2 = build_model(base_model)
resnet_model_2.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Phase 1: Train only top layers
resnet_model_2.fit(train_img, validation_data=val_image, epochs=5, callbacks=[early_stop, reduce_lr])

# 2. Unfreeze last 30 layers
for layer in base_model.layers[-10:]:
    layer.trainable = True

resnet_model_2.compile(optimizer=Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Phase 2: Fine-tune
resnet_model_2.fit(train_img, validation_data=val_image, epochs=5, callbacks=[early_stop, reduce_lr])


Epoch 1/5
266/266 ━━━━━━━━━━━━━━━━━━━━ 386s 1s/step - accuracy: 0.0831 - loss: 3.0281 - val_accuracy: 0.1522 - val_loss: 2.4874 - learning_rate: 1.0000e-04
Epoch 2/5
266/266 ━━━━━━━━━━━━━━━━━━━━ 0s 957ms/step - accuracy: 0.1387 - loss: 2.5935